In [24]:
using ITensors

In [25]:
"""
Synthesize two MPO as M2 * M1.
When using the synthesized MPO, M1 will be applied to a MPS first, then M2.
"""
#Not used
#function synthesize(M1::MPO, M2::MPO)
#    M21 = contract(prime(M2), M1)
#    prime(M21, -1, plev=2) # -1? plev=2?
#end

#use apply

"Synthesize two MPO as M2 * M1.\nWhen using the synthesized MPO, M1 will be applied to a MPS first, then M2.\n"

In [29]:
function ITensors.op(::OpName"CPHASE2", ::SiteType"Qubit"; ϕ::Number, inv=false)
  if inv
    sgn = -1.0
  else
    sgn = 1.0
  end
  return [
      1 0 0 0
      0 1 0 0
      0 0 1 0
      0 0 0 exp(sgn * im * 2π * ϕ)
    ]
  end
  ITensors.op(::OpName"Cphase2", t::SiteType"Qubit"; kwargs...) = op("CPHASE2", t; kwargs...)

  # https://github.com/ITensor/ITensors.jl/blob/main/src/physics/site_types/qubit.jl

## QFT FUNCTION
### 繰り返し圧縮をする場合 

In [39]:
#  One Controled Phase gate
# controled がかかっていない量子ビットには、Identityを作用させる。
function CP(nqubit, sites, a, b, phase; inv=true)
    cp = op("CPHASE2", sites, a, b; ϕ=1/2^(phase), inv)
    for i in 1:nqubit #[1,2]から1,2を抜いた配列
        if i == a || i == b
            continue
        end
        #@show i
        cp *= op("Id", sites, i)
    end
    return cp
end

# layer1 のcontroled phase gate ######
function MakeOneBlockInvQFT(nqubit, sites, offset; inv=true, cutoffcp::Float64=1e-14) #offsetは,Hがかかる位置
    #Dummy
    block = MPO(Op("Id", offset), sites) 
    for i in reverse(range(1, nqubit-offset)) #EX)nqubit=3, offset=1
        # range(1, 0)の場合for分の中で何も起こらないことを利用している。
        phase = i+1
        cp = CP(nqubit, sites, i+offset, offset, phase, inv=inv) #phase=2,3,4,5,6,,,　#layerをここにする。
        cp = MPO(cp, sites)
        block = apply(cp, block; cutoff=cutoffcp)
    end

    h = MPO(Op("H", offset), sites)
    block = apply(h, block, cutoff=cutoffcp)
    return block
end

function ExecFullInvQFT(sites, nqubit; cutoffcp::Float64=1e-14, cutoffblock::Float64=1e-20)
    fullqft = MPO(Op("Id", 1), sites) #Dummy
    for offset in reverse(range(1, nqubit))
        qftblock = MakeOneBlockInvQFT(nqubit, sites, offset; cutoffcp=cutoffcp)
        fullqft = apply(qftblock, fullqft;  cutoffblock=1e-20) # Compress!
    end
    return fullqft
end

ExecFullInvQFT (generic function with 3 methods)

## test

## 1量子ビット逆QFT

In [40]:
## check
nqubit = 1
sites = siteinds("Qubit", nqubit)  
psi = MPS(sites, ["1"]) 
#qft_1 = ExecFullInvQFT(sites, nqubit; cutoffcp=1e-14, cutoffblock=1e-14)
qft_1 = ExecFullInvQFT(sites, nqubit; cutoffcp=1e-14,  cutoffblock=1e-20)
qft_1_mpo = apply(qft_1, psi; cutoff=1e-20) 
qft_1_arr = Array(reduce(*, qft_1_mpo), sites)
@show qft_1_arr

## test
nqubit = 1 
sites = siteinds("Qubit", nqubit)  
layer1 = MPO(Op("H", 1), sites)
psi = MPS(sites, ["1"]) 
res = apply(layer1, psi; cutoff=1e-20) 
res_arr = Array(reduce(*, res), sites)

@assert qft_1_arr == res_arr

qft_1_arr = [0.7071067811865475, -0.7071067811865475]


## 2量子ビット逆QFT

In [41]:
## check
nqubit = 2
sites = siteinds("Qubit", nqubit)  
psi = MPS(sites, ["1","1"]) 
qft_2 = ExecFullInvQFT(sites, nqubit)
qft_2_mpo = apply(qft_2, psi; cutoff=1e-20) 
qft_2_arr = Array(reduce(*, qft_2_mpo), sites)
@show qft_2_arr

## test
nqubit = 2 
sites = siteinds("Qubit", nqubit)  
psi = MPS(sites, ["1", "1"]) 
# Create H_1
h1 = MPO(Op("H", 2), sites)
# Create CR(2,1)
cp12 = op("CPHASE2", sites, 2,1,inv=true; ϕ=(1/2^2))
cp12 = MPO(cp12, sites)
layer1 = apply(cp12, h1)
h2 = MPO(Op("H", 1), sites)
layer2 = apply(h2, layer1)
res = apply(layer2, psi; cutoff=1e-20) 
res_arr = Array(reduce(*, res), sites)
@show res_arr

@assert isapprox(real(res_arr), real(qft_2_arr))
@assert isapprox(imag(res_arr), imag(qft_2_arr))

qft_2_arr = ComplexF64[0.4999999999999994 + 1.867230990547788e-17im -9.717693283966666e-17 + 0.49999999999999983im; -0.4999999999999994 - 1.867230990547788e-17im 9.717693283966666e-17 - 0.49999999999999983im]
res_arr = ComplexF64[0.49999999999999933 + 3.852037975195809e-16im 3.1597621837087197e-16 + 0.4999999999999994im; -0.49999999999999933 - 3.852037975195809e-16im -3.1597621837087197e-16 - 0.4999999999999994im]


## 3量子ビット

In [42]:
## check
nqubit = 3
sites = siteinds("Qubit", nqubit)  
psi = MPS(sites, ["1","1","0"]) 
qft_3 = ExecFullInvQFT(sites, nqubit)
qft_3_mpo = apply(qft_3, psi; cutoff=1e-20) 
qft_3_arr = Array(reduce(*, qft_3_mpo), sites)
@show qft_3_arr


##test
nqubit = 3
sites = siteinds("Qubit", nqubit) 
# layer1
h3 = MPO(Op("H", 3), sites)
#layer2

cp23 = op("CPHASE2", sites, 3,2; ϕ=(1/2^2),inv=true) * op("Id", sites, 1)
cp23 = MPO(cp23, sites)
h2 = MPO(Op("H", 2), sites)
layer2 = apply(h2, cp23)
#layer3
cp13 = op("CPHASE2", sites, 3,1; ϕ=(1/2^3),inv=true) * op("Id", sites, 2)
cp13 = MPO(cp13, sites)
cp12 = op("CPHASE2", sites, 2,1; ϕ=(1/2^2),inv=true) * op("Id", sites, 3)
cp12 = MPO(cp12, sites)
h1 = MPO(Op("H", 1), sites)
layer3 = apply(cp12, cp13) # H_1 
layer3 = apply(h1, layer3)

#layer3
layer12 = apply(layer2, h3)
layer123 = apply(layer3, layer12)
psi = MPS(sites, ["1", "1", "0"]) 
res = apply(layer123, psi; cutoff=1e-20) 
res_arr = Array(reduce(*, res), sites) #c_{ijk}
@show res_arr

@assert isapprox(real(res_arr), real(qft_3_arr))
@assert isapprox(imag(res_arr), imag(qft_3_arr))


qft_3_arr = [0.35355339059327306 - 1.5027461392960995e-16im 3.019440158025344e-16 + 0.35355339059327295im; -0.3535533905932732 + 9.476346269835214e-17im -2.4643286457127655e-16 - 0.35355339059327306im;;; -0.2499999999999998 - 0.24999999999999956im 0.24999999999999936 - 0.24999999999999983im; 0.24999999999999983 + 0.2499999999999997im -0.24999999999999947 + 0.2499999999999999im]
res_arr = [0.3535533905932729 - 9.795720772389027e-17im 4.949868246864862e-16 + 0.3535533905932727im; -0.3535533905932729 + 9.415106090377989e-17im -4.4186448783088004e-16 - 0.35355339059327273im;;; -0.24999999999999906 - 0.24999999999999933im 0.24999999999999892 - 0.24999999999999978im; 0.24999999999999906 + 0.24999999999999933im -0.249999999999999 + 0.24999999999999978im]
